In [25]:
# Copyright (c) Facebook, Inc. and its affiliates.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Bert Pipeline : PyTorch BERT News Classfication

This notebook shows PyTorch BERT end-to-end news classification example using Kubeflow Pipelines.


An example notebook that demonstrates how to:

* Get different tasks needed for the pipeline
* Create a Kubeflow pipeline
* Include Pytorch KFP components to preprocess, train, visualize and deploy the model in the pipeline
* Submit a job for execution
* Query(prediction and explain) the final deployed model
* Interpretation of the model using the Captum Insights


In [3]:
! pip uninstall -y kfp
! pip install --no-cache-dir kfp

Found existing installation: kfp 1.0.4
Uninstalling kfp-1.0.4:
  Successfully uninstalled kfp-1.0.4
     |████████████████████████████████| 225 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 83.5 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 47.2 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 76.8 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 71.1 MB/s eta 0:00:01
  Created wheel for kfp: filename=kfp-1.6.4-py3-none-any.whl size=307978 sha256=ddf494e3f0dd0c06877b7a93c5c54eed26b94e29460c227f28a66225cc282383
  Stored in directory: /tmp/pip-ephem-wheel-cache-rnckkenu/wheels/94/27/d3/4e61074deae41e0ef5f166343fc7540eb85536c3d49f4cd25f
  Created wheel for docstring-parser: filename=docstring_parser-0.8.1-py3-none-any.whl size=19

In [1]:
import kfp
import json
import os
from kfp.onprem import use_k8s_secret
from kfp import components
from kfp.components import load_component_from_file, load_component_from_url, InputPath
from kfp import dsl
from kfp import compiler

kfp.__version__

'1.6.4'

# Enter your gateway and the cookie
[Use this extension on chrome to get token]( https://chrome.google.com/webstore/detail/editthiscookie/fngmhnnpilhplaeedifhccceomclgfbg?hl=en)

![image.png](./image.png)

## Update values for the ingress gateway and auth session

In [2]:
INGRESS_GATEWAY='http://istio-ingressgateway.istio-system.svc.cluster.local'
AUTH="MTYyNDYxNTY2M3xOd3dBTkVkSlUxbFhUVlJKV2xNM01qWklSREpKTlRWSVJGRkxXbFJSVWtaRVdFeEdRa3RMV2s1TE4wRk5XRXRPVGxSUlZqWkRVRUU9fPKmcl46SVyEzMqhTl9bnDPZBnGyU1Sg5ipjOQawdCN9"
NAMESPACE="kubeflow-user-example-com"
COOKIE="authservice_session="+AUTH
EXPERIMENT="Default"
dist_volume = 'dist-vol'

## Set Log bucket and Tensorboard Image

In [3]:
MINIO_ENDPOINT="http://minio-service.kubeflow:9000"
LOG_BUCKET="mlpipeline"
TENSORBOARD_IMAGE="public.ecr.aws/y1x1p2u5/tboard:latest"

In [4]:
client = kfp.Client(host=INGRESS_GATEWAY+"/pipeline", cookies=COOKIE)

In [5]:
client.create_experiment(EXPERIMENT)
experiments = client.list_experiments(namespace=NAMESPACE)
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 6, 21, 13, 13, 6, tzinfo=tzlocal()),
 'description': None,
 'id': 'ba9b7266-2b1c-4729-afcd-be808c25c5af',
 'name': 'Default',
 'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                  'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

## Set Inference parameters

In [6]:
DEPLOY_NAME="bertserve"
MODEL_NAME="bert"

In [11]:
from kubernetes.client.models import V1Volume, V1PersistentVolumeClaimVolumeSource

def create_dist_pipeline():
    kubernetes_create_pvc_op(pvc_name=dist_volume, storage_size= "20Gi", namespace=NAMESPACE)

create_volume_run = client.create_run_from_pipeline_func(create_dist_pipeline, arguments={})
create_volume_run.wait_for_run_completion()

{'pipeline_runtime': {'pipeline_manifest': None,
                      'workflow_manifest': '{"metadata":{"name":"create-dist-pipeline-ht86q","generateName":"create-dist-pipeline-","namespace":"kubeflow-user-example-com","selfLink":"/apis/argoproj.io/v1alpha1/namespaces/kubeflow-user-example-com/workflows/create-dist-pipeline-ht86q","uid":"e806eda5-0409-4487-ac96-cc297da737fc","resourceVersion":"1678547","generation":3,"creationTimestamp":"2021-06-25T10:10:46Z","labels":{"pipeline/runid":"c3bb05db-e555-4583-90e4-346adb2d9cad","pipelines.kubeflow.org/kfp_sdk_version":"1.6.4","workflows.argoproj.io/completed":"true","workflows.argoproj.io/phase":"Succeeded"},"annotations":{"pipelines.kubeflow.org/kfp_sdk_version":"1.6.4","pipelines.kubeflow.org/pipeline_compilation_time":"2021-06-25T10:10:46.639252","pipelines.kubeflow.org/pipeline_spec":"{\\"name\\": '
                                           '\\"Create dist '
                                           'pipeline\\"}","pipelines.kubefl

In [12]:
prepare_tensorboard_op = load_component_from_file("common/tensorboard/component.yaml")
prep_op = components.load_component_from_file(
    "bert/yaml/pre_process/component.yaml"
)
train_op = components.load_component_from_file(
    "bert/yaml/train/component.yaml"
)
deploy_op = load_component_from_file("common/deploy/component.yaml")
minio_op = components.load_component_from_file(
    "common/minio/component.yaml"
)
kubernetes_create_pvc_op = load_component_from_file('volume/component.yaml')
pytorch_job_op = load_component_from_file(
    "common/pytorch_job/component.yaml"
)
cp_op = load_component_from_file(
    "copy/component.yaml"
)
ls_op = load_component_from_file(
    "list/component.yaml"
)

## Define pipeline

In [88]:
@dsl.pipeline(name="Training pipeline", description="Sample training job test")
def pytorch_bert(
    minio_endpoint=MINIO_ENDPOINT,
    log_bucket=LOG_BUCKET,
    log_dir=f"/tensorboard/logs/{dsl.RUN_ID_PLACEHOLDER}",
    mar_path=f"mar/{dsl.RUN_ID_PLACEHOLDER}/model-store",
    config_prop_path=f"mar/{dsl.RUN_ID_PLACEHOLDER}/config",
    model_uri=f"s3://mlpipeline/mar/{dsl.RUN_ID_PLACEHOLDER}",
    tf_image=TENSORBOARD_IMAGE,
    deploy=DEPLOY_NAME,
    namespace=NAMESPACE,
    confusion_matrix_log_dir=f"confusion_matrix/{dsl.RUN_ID_PLACEHOLDER}/",
    num_samples=1000
):
    
  
    prepare_tb_task = prepare_tensorboard_op(
        log_dir_uri=log_dir,
        image=tf_image
    ).set_display_name("Visualization")

    prep_task = prep_op().after(prepare_tb_task).set_display_name("Preprocess & Transform")
    prep_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    copy_task = cp_op(prep_task.outputs['output_data'], "/model/dataset").add_pvolumes({"/model": dsl.PipelineVolume(pvc=dist_volume)}).after(prep_task)
    copy_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    ls_task = ls_op("/model").add_pvolumes({"/model": dsl.PipelineVolume(pvc=dist_volume)}).after(copy_task)
    ls_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    train_task = pytorch_job_op(
        name="pytorch-bert", 
        namespace=namespace, 
        master_spec=
        '''
        {
          "replicas": 1,
          "imagePullPolicy": "Always",
          "restartPolicy": "OnFailure",
          "template": {
            "metadata": {
              "annotations": {
                "sidecar.istio.io/inject": "false"
              }
            },
            "spec": {
              "containers": [
                {
                  "name": "pytorch",
                  "image": jagadeeshj/dist_bert:v1.e,
                  "args": [
                    "--profiler",  "pytorch",
                    "--dataset_path", "/model/dataset",
                    "--num_samples", "1000",
                    "--checkpoint_dir", "/model/checkpoint",
                    "--gpus", "-1",
                    "--accelerator", "ddp",
                    "--num_nodes", "2"
                  ],
                  "ports": [
                    {
                      "containerPort": 24456,
                      "name": "pytorchjob-port"
                    }
                  ],
                  "resources": {
                    "limits": {
                      "nvidia.com/gpu": 4
                    }
                  },
                  "volumeMounts": [
                    {
                      "mountPath": "/model",
                      "name": "model-volume"
                    }
                  ]
                }
              ],
              "volumes": [
                {
                  "name": "model-volume",
                  "persistentVolumeClaim": {
                    "claimName": "dist-vol"
                  }
                }
              ]
            }
          }
        }
        ''', 
        worker_spec=
        '''
        {
          "replicas": 1,
          "imagePullPolicy": "Always",
          "restartPolicy": "OnFailure",
          "template": {
            "metadata": {
              "annotations": {
                "sidecar.istio.io/inject": "false"
              }
            },
            "spec": {
              "containers": [
                {
                  "name": "pytorch",
                  "image": jagadeeshj/dist_bert:v1.e,
                  "args": [
                    "--profiler",  "pytorch",
                    "--dataset_path", "/model/dataset",
                    "--num_samples", "1000",
                    "--checkpoint_dir", "/model/checkpoint",
                    "--gpus", "-1",
                    "--accelerator", "ddp",
                    "--num_nodes", "2"
                  ],
                  "ports": [
                    {
                      "containerPort": 24456,
                      "name": "pytorchjob-port"
                    }
                  ],
                  "resources": {
                    "limits": {
                      "nvidia.com/gpu": 4
                    }
                  },
                  "volumeMounts": [
                    {
                      "mountPath": "/model",
                      "name": "model-volume"
                    }
                  ]
                }
              ],
              "volumes": [
                {
                  "name": "model-volume",
                  "persistentVolumeClaim": {
                    "claimName": "dist-vol"
                  }
                }
              ]
            }
          }
        }
        '''
    ).after(ls_task)

    
#     train_task = (
#         train_op(
#             input_data=prep_task.outputs["output_data"],
#             profiler="pytorch",
#             confusion_matrix_url=f"minio://{log_bucket}/{confusion_matrix_log_dir}",
#             num_samples=num_samples,
#             # For GPU set gpu count and accelerator type
#             gpus=0,
#             accelerator='None'
#         )
#         .after(prep_task)
#         .set_display_name("Training")
#     )
    # For GPU uncomment below line and set GPU limit and node selector
    # ).set_gpu_limit(1).add_node_selector_constraint('cloud.google.com/gke-accelerator','nvidia-tesla-p4')

#     minio_tb_upload = (
#         minio_op(
#             bucket_name="mlpipeline",
#             folder_name=lThe volume doog_dir,
#             input_path=train_task.outputs["tensorboard_root"],
#             filename="",
#         )
#         .after(train_task)
#         .set_display_name("Tensorboard Events Pusher")
#     )
#     minio_mar_upload = (
#         minio_op(
#             bucket_name="mlpipeline",
#             folder_name=mar_path,
#             input_path=train_task.outputs["checkpoint_dir"],
#             filename="bert_test.mar",
#         )
#         .after(train_task)
#         .set_display_name("Mar Pusher")
#     )
#     minio_config_upload = (
#         minio_op(
#             bucket_name="mlpipeline",
#             folder_name=config_prop_path,
#             input_path=train_task.outputs["checkpoint_dir"],
#             filename="config.properties",
#         )
#         .after(train_task)
#         .set_display_name("Conifg Pusher")
#     )

#     model_uri = str(model_uri)
#     isvc_yaml = """
#     apiVersion: "serving.kubeflow.org/v1beta1"
#     kind: "InferenceService"
#     metadata:
#       name: {}
#       namespace: {}
#     spec:
#       predictor:
#         serviceAccountName: sa
#         pytorch:
#           storageUri: {}
#           resources:
#             limits:
#               memory: 4Gi   
#     """.format(
#         deploy, namespace, model_uri
#     )

#     # For GPU inference use below yaml with gpu count and accelerator
#     gpu_count = "1"
#     accelerator = "nvidia-tesla-p4"
#     isvc_gpu_yaml = """
#     apiVersion: "serving.kubeflow.org/v1beta1"
#     kind: "InferenceService"
#     metadata:
#       name: {}
#       namespace: {}
#     spec:
#       predictor:
#         serviceAccountName: sa
#         pytorch:
#           storageUri: {}
#           resources:
#             limits:
#               memory: 4Gi   
#               nvidia.com/gpu: {}
#           nodeSelector:
#             cloud.google.com/gke-accelerator: {}
# """.format(
#         deploy, namespace, model_uri, gpu_count, accelerator
#     )
#     # Update inferenceservice_yaml for GPU inference
#     deploy_task = (
#         deploy_op(action="apply", inferenceservice_yaml=isvc_yaml)
#         .after(minio_mar_upload)
#         .set_display_name("Deployer")
#     )

    dsl.get_pipeline_conf().add_op_transformer(
        use_k8s_secret(
            secret_name="mlpipeline-minio-artifact",
            k8s_secret_key_to_env={
                "secretkey": "MINIO_SECRET_KEY",
                "accesskey": "MINIO_ACCESS_KEY",
            },
        )
    )
#     kfp.dsl.get_pipeline_conf().data_passing_method = volume_based_data_passing_method

In [89]:
pipeline_conf = kfp.dsl.PipelineConf()
# pipeline_conf.data_passing_method = volume_based_data_passing_method

In [90]:
# Compile pipeline
compiler.Compiler().compile(pytorch_bert, 'pytorch.tar.gz',  pipeline_conf=pipeline_conf, type_check=True)

In [91]:
# Execute pipeline
run = client.run_pipeline(my_experiment.id, 'pytorch-bert', 'pytorch.tar.gz')

## Wait for inference service below to go to `READY True` state.

In [13]:
!kubectl get isvc $DEPLOY

NAME         URL                                                       READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                  AGE
bertserve    http://bertserve.kubeflow-user-example-com.example.com    True           100                              bertserve-predictor-default-zckhh    44h
torchserve   http://torchserve.kubeflow-user-example-com.example.com   True           100                              torchserve-predictor-default-tdknk   47h


# Get Inferenceservice name

In [14]:
INFERENCE_SERVICE_LIST = ! kubectl get isvc {DEPLOY_NAME} -n {NAMESPACE} -o json | python3 -c "import sys, json; print(json.load(sys.stdin)['status']['url'])"| tr -d '"' | cut -d "/" -f 3
INFERENCE_SERVICE_NAME = INFERENCE_SERVICE_LIST[0]
INFERENCE_SERVICE_NAME

'bertserve.kubeflow-user-example-com.example.com'

# Prediction Request

In [27]:
!curl -v -H "Host: $INFERENCE_SERVICE_NAME" -H "Cookie: $COOKIE" "$INGRESS_GATEWAY/v1/models/$MODEL_NAME:predict" -d @./bert/sample.txt > bert_prediction_output.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:predict HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMzI1NDI0NHxOd3dBTkVGU1UxaElXRXN5VUVKTVJrZFVUMWhDU1VoVlNVMUhSRFZaVVRWQlNrVkhORTAzUTFWTFVqZExSa0pHVmpWU016SmFOa0U9fO86sBQIDoqYUxX9ffUnG7xS8xyysaWppWJa0c3QBRJd
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
} [84 bytes data]
* upload completely sent off: 84 out of 84 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 33
< content-type: application/json; charset=UTF-8
< date: Thu, 10 Jun 202

In [16]:
! cat bert_prediction_output.json

{"predictions": ["\"Sci/Tech\""]}

# Explanation Request

In [17]:
!curl -v -H "Host: $INFERENCE_SERVICE_NAME" -H "Cookie: $COOKIE" "$INGRESS_GATEWAY/v1/models/$MODEL_NAME:explain" -d @./bert/sample.txt  > bert_explaination_output.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:explain HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMzI1NDI0NHxOd3dBTkVGU1UxaElXRXN5VUVKTVJrZFVUMWhDU1VoVlNVMUhSRFZaVVRWQlNrVkhORTAzUTFWTFVqZExSa0pHVmpWU016SmFOa0U9fO86sBQIDoqYUxX9ffUnG7xS8xyysaWppWJa0c3QBRJd
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
} [84 bytes data]
* upload completely sent off: 84 out of 84 bytes
100    84    0     0  100    84      0      1  0:01:24  0:00:52  0:00:32     000:02     00:30  0:00:12     08  0:00:04     0  0   0      1  0:01:24  0:

In [18]:
! cat bert_explaination_output.json

{"explanations": [{"words": ["[CLS]", "bloomberg", "has", "reported", "on", "the", "economy", "[SEP]"], "importances": [0.49803253502586686, -0.042289041470624116, -0.2269101439114476, 0.15573707990586028, 0.0867725310070807, 0.17919607383818434, 0.5255456841947312, -0.5988271940782108], "delta": 0.12081503337965546}]}

In [19]:
explanations_json = json.loads(open("./bert_explaination_output.json", "r").read())
explanations_json

{'explanations': [{'words': ['[CLS]',
    'bloomberg',
    'has',
    'reported',
    'on',
    'the',
    'economy',
    '[SEP]'],
   'importances': [0.49803253502586686,
    -0.042289041470624116,
    -0.2269101439114476,
    0.15573707990586028,
    0.0867725310070807,
    0.17919607383818434,
    0.5255456841947312,
    -0.5988271940782108],
   'delta': 0.12081503337965546}]}

In [20]:
prediction_json = json.loads(open("./bert_prediction_output.json", "r").read())

In [21]:
import torch
attributions = explanations_json["explanations"][0]['importances']
tokens = explanations_json["explanations"][0]['words']
delta = explanations_json["explanations"][0]['delta']

attributions = torch.tensor(attributions)
pred_prob = 0.75
pred_class = prediction_json["predictions"][0]
true_class = "Business"
attr_class ="world"

# Visualization of Predictions

In [23]:
from captum.attr import visualization
vis_data_records =[]
vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred_prob,
                            pred_class,
                            true_class,
                            attr_class,
                            attributions.sum(),       
                            tokens,
                            delta))

In [24]:
vis = visualization.visualize_text(vis_data_records)

### visualization appreas as below
![viz1.png](./viz1.png)

## Cleanup Script

In [10]:
! kubectl delete --all isvc -n $NAMESPACE

In [11]:
! kubectl delete pod --field-selector=status.phase==Succeeded -n $NAMESPACE